In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os                                                                                                                                                                                        
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
;
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read meta data (ใช้รูป crop)
metadata = pd.read_csv('../input/font-detection-dataset-crop/label_train.csv')

In [ ]:
# add .png to make datagenerator  read file 
metadata['parentId']  = metadata['parentId'] + '.png'

In [ ]:
# Check for class number
print('Num font:'+ str(len(metadata['fontFamily'].value_counts())))
print('Num size:'+ str(len(metadata['fontSize'].value_counts())))
print('Num weight:'+ str(len(metadata['fontWeight'].value_counts())))
print('Num style:'+ str(len(metadata['fontStyle'].value_counts())))

In [ ]:
# map Fontfamily to integer
unique_vals = metadata['fontFamily'].unique()
metadata['fontFamily_int'] = metadata['fontFamily'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))
'''
# map fontStyle integer
unique_vals = metadata['fontStyle'].unique()
metadata['fontStyle_int'] = metadata['fontStyle'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))

# map fontWeight integer
unique_vals = metadata['fontWeight'].unique()
metadata['fontWeight_int'] = metadata['fontWeight'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))

# map fontSize integer
unique_vals = metadata['fontSize'].unique()
metadata['fontSize_int'] = metadata['fontSize'].replace(to_replace=unique_vals,
           value= list(range(len(unique_vals))))
'''


In [ ]:
metadata

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import BatchNormalization , Dense , Dropout , GlobalAveragePooling2D

In [ ]:
#define backbone model
def createInceptionV3(input_shape):
    model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=input_shape,
    pooling=None)
    return model

In [ ]:
# declare backbone model
basemodel = createInceptionV3((105, 105, 3))
basemodel.trainable = True 

In [ ]:
# Top layer for 4 task
def create_model(base_model):
    
    x =  BatchNormalization()(base_model.output)
    x =  GlobalAveragePooling2D()(x)
    
    # Font Family
    dense_family = Dense(1024,activation = 'relu')(x)
    dropout_family = Dropout(0.2)(dense_family)
    output_family = Dense(10,activation = 'softmax')(dropout_family)
 
    model = tf.keras.Model(inputs = base_model.input , outputs = output_family)
    
    return model
    

In [ ]:
# Create model
model = create_model(basemodel)

In [ ]:
#model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
num_layer = len(model.layers)
print(num_layer)
print(len(model.trainable_variables))

In [ ]:
model.compile(optimizer =  tf.keras.optimizers.Adam(),
              # กำหนด loss แบบ list ตาม order ที่create model or use 1 for all
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = ['acc']
            )

In [ ]:
'''
def resize_image(image):
    image = image.copy()
    width = 299
    heigth = 299
    for i in range(int(width/image.shape[1])) :
        if(image.shape[1] < width ):
            image = np.hstack((image,image))
        else:
            break
    for i in range(int(heigth/image.shape[0])) :
        if(image.shape[0] < heigth ):
            image = np.vstack((image,image))
        else:
            break
    image = image[0:heigth,0:width].copy()
    return image
path = "../input/fontregcogappman/font_detection_dataset/font_detection_dataset/"
import glob
# All files and directories ending with .txt and that don't begin with a dot:
files_name_train = (glob.glob(path + "image_train/*"))
files_name_test = (glob.glob(path + "image_test/*"))
print(files_name_train[0:2])
import cv2
import matplotlib.pyplot as plt
for file_path in files_name_train:
    image = cv2.imread(file_path)
    image = resize_image(image)
    cv2.imwrite(file_path, image)
for file_path in files_name_test:
    image = cv2.imread(file_path)
    image = resize_image(image)
    cv2.imwrite(file_path, image)
'''

In [ ]:
# Create DATA PIPELINE 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATA_PATH = '../input/font-detection-dataset-crop/'

# use preprocessing_function match with model
image_generator = ImageDataGenerator(preprocessing_function = tf.keras.applications.inception_v3.preprocess_input)
train_generator = image_generator.flow_from_dataframe(metadata[:-10000],# edit for metadata[:-10000] for real data
                                    directory = DATA_PATH + 'image_train',
                                    x_col ='parentId',
                                    class_mode = 'raw',
                                    y_col = 'fontFamily_int',
                                    batch_size = 128,
                                    seed = 69,
                                    shuffle = True,
                                    target_size=(105,105)
                                    )
validation_generator = image_generator.flow_from_dataframe(metadata[-10000:],# edit for metadata[-10000:] for real data
                                    directory = DATA_PATH + 'image_train',
                                    x_col ='parentId',
                                    class_mode = 'raw',
                                    y_col = 'fontFamily_int',
                                    batch_size = 128,
                                    seed = 69,
                                    shuffle = True,
                                    target_size=(105,105)
                                    )


In [ ]:
# Testing purpose
#import matplotlib.pyplot as plt

#plt.imshow(next(iter(train_generator))[0][0])
#print(next(iter(train_generator))[0][0])
'''
ori_img = plt.imread('../input/fontregcogappman/font_detection_dataset/font_detection_dataset/image_train/100045cf.png')
plt.imshow(ori_img)
img = resize_image(ori_img)
plt.imshow(img)
'''

In [ ]:
model.load_weights('../input/outputfontreg-1/inception_best.h5')

In [ ]:
# Create Checkpoint , Callback
from tensorflow.keras.callbacks import Callback , EarlyStopping , ModelCheckpoint , ReduceLROnPlateau

checkpoint = ModelCheckpoint( filepath = './inception_best.h5', monitor='val_loss', verbose=0, save_best_only=True,
                            save_weights_only=True, mode='auto', save_freq='epoch')
earlystop =  EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0,
                            mode='auto', baseline=None, restore_best_weights=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min')

callbacks = [checkpoint,earlystop,reduce_lr]

In [ ]:
# fit model
history = model.fit(x=train_generator,
                    validation_data=validation_generator,
                    epochs=25,
                    callbacks=callbacks)